In [1]:
# !pip install -U langchain
# !pip install -U langchain-community
# !pip install -U chromadb
# !pip install -U sentence-transformers tqdm
# !pip install -U datasets
# !pip  install -U langchain_experimental
# !pip install ragas


In [2]:
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.document_loaders import WebBaseLoader
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
import os
import torch
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    SentenceTransformersTokenTextSplitter,
    TextSplitter,
    TokenTextSplitter,
)
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from tqdm import tqdm
import pandas as pd
import time
import logging
from datasets import Dataset

import os
import torch
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    SentenceTransformersTokenTextSplitter,
    TextSplitter,
    TokenTextSplitter,
)
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from tqdm import tqdm
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    answer_correctness
)
from ragas import evaluate
import pandas as pd
import time
import logging
from datasets import Dataset
import chromadb
from chromadb.config import Settings

USER_AGENT environment variable not set, consider setting it to identify your requests.
/home/ubuntu/itmo_productrag/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import hashlib

file_path = '../data/hmao_npa_cleaned_v2.txt'
db_dir = '../db'

f = open(file_path, "rb")

def calculate_sha256(file):
    sha256 = hashlib.sha256()
    for chunk in iter(lambda: file.read(8192), b""):
        sha256.update(chunk)
    return sha256.hexdigest()


collection_name = 'modelv1_750_100_base'

# Read the text content from the file
loader = TextLoader(file_path)
documents = loader.load()

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "ianssens/e5-model-rag"
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

# Initialize embeddings
embedding_model = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={'device': device}
)

client = chromadb.HttpClient(
        host='localhost',
        port=8000,
        ssl=False,
        headers=None,
        settings=Settings(),
        tenant=chromadb.DEFAULT_TENANT,
        database=chromadb.DEFAULT_DATABASE,
    )


# Create Langchain Chroma wrapper
db = Chroma(
    client=client,
    collection_name=collection_name,
    embedding_function=embedding_model
)

cuda


No sentence-transformers model found with name ianssens/e5-model-rag. Creating a new one with mean pooling.
/tmp/ipykernel_736592/193634173.py:26: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  db = Chroma(


In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tqdm import tqdm  # Add tqdm for progress tracking

# Assuming 'documents' is a list of your documents
rec_char_splitter = RecursiveCharacterTextSplitter(chunk_size=750, chunk_overlap=100)
docs = rec_char_splitter.split_documents(documents)

# Use tqdm to add a progress bar for adding documents
for doc in tqdm(docs, desc="Adding documents"):
    db.add_documents([doc])

print(f"Connected to Chroma collection: {collection_name}")
print(f"Number of documents in collection: {db._collection.count()}")


Adding documents: 100%|██████████| 33789/33789 [22:29<00:00, 25.04it/s]


Connected to Chroma collection: modelv1_750_100_base
Number of documents in collection: 33789


In [6]:
def query_vector_store(
    store_name, query, embedding_function, search_type, search_kwargs
):
      print(f"\n--- Querying the Vector Store {store_name} ---")
      retriever = db.as_retriever(
          search_type=search_type,
          search_kwargs=search_kwargs,
      )
      relevant_docs = retriever.invoke(query)
      # Display the relevant results with metadata
      print(f"\n--- Relevant Documents for {store_name} ---")
      for i, doc in enumerate(relevant_docs, 1):
          print(f"Document {i}:\n{doc.page_content}\n")
          if doc.metadata:
              print(f"Source: {doc.metadata.get('source', 'Unknown')}\n")


query = "Как осуществляется представление информации в ходе проверки юридического лица или индивидуального предпринимателя?"

query_vector_store(
    collection_name,
    query,
    embedding_model,
    "similarity",
    {"k": 5},
)


--- Querying the Vector Store modelv1_750_100_base ---

--- Relevant Documents for modelv1_750_100_base ---
Document 1:
уполномоченным представителем юридического лица, индивидуальным предпринимателем или уполномоченным представителем индивидуального предпринимателя определяют круг лиц, с которыми будет осуществляться взаимодействие в ходе проверки, уточняет перечень документов, возможность с их ознакомлением, а также временной режим проверки (с учетом действующего режима работы юридического лица, индивидуального предпринимателя). 3.12.5. В ходе проверки осуществляются: визуальный осмотр территории, находящейся в пользовании юридического лица, индивидуального предпринимателя; анализ документов и представленной информации; иные мероприятия по надзору, предусмотренные Федеральным законом отдекабрягода N 294-ФЗ. 3.12.5.1. Визуальный осмотр территории,

Source: ../data/hmao_npa_cleaned_v2.txt

Document 2:
лицу или уполномоченному представителю юридического лица, индивидуальному предприним

In [7]:
import os

from dotenv import load_dotenv
from langchain_community.vectorstores import Chroma
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI, OpenAIEmbeddings


query = "Как осуществляется представление информации в ходе проверки юридического лица или индивидуального предпринимателя?"

# Retrieve relevant documents based on the query
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)
relevant_docs = retriever.invoke(query)

# Display the relevant results with metadata
print("\n--- Relevant Documents ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"Document {i}:\n{doc.page_content}\n")

# Combine the query and the relevant document contents
combined_input = (
    "Вот несколько документов, которые могут помочь ответить на вопрос: "
    + query
    + "\n\nСоответствующие документы:\n"
    + "\n\n".join([doc.page_content for doc in relevant_docs])
    + "\n\nПожалуйста, предоставьте ответ, основываясь только на предоставленных документах. Если ответ не найден в документах, ответьте 'Я не уверен'."
)

model = ChatOpenAI(model="gpt-4o-mini", api_key="sk-or-vv-678cbad362790622377958c8beb4937642cfb7083ee6cfce2d62886dc4b86326", base_url="https://api.vsegpt.ru/v1")
messages = [
    SystemMessage(content="Ты ассистент отвечающий на вопросы по информации которая есть в нормативно-правовых-актах"),
    HumanMessage(content=combined_input),
]

# Invoke the model with the combined input
result = model.invoke(messages)

# Display the full result and content only
print("\n--- Generated Response ---")
# print("Full result:")
# print(result)
print("Content only:")
print(result.content)


--- Relevant Documents ---
Document 1:
уполномоченным представителем юридического лица, индивидуальным предпринимателем или уполномоченным представителем индивидуального предпринимателя определяют круг лиц, с которыми будет осуществляться взаимодействие в ходе проверки, уточняет перечень документов, возможность с их ознакомлением, а также временной режим проверки (с учетом действующего режима работы юридического лица, индивидуального предпринимателя). 3.12.5. В ходе проверки осуществляются: визуальный осмотр территории, находящейся в пользовании юридического лица, индивидуального предпринимателя; анализ документов и представленной информации; иные мероприятия по надзору, предусмотренные Федеральным законом отдекабрягода N 294-ФЗ. 3.12.5.1. Визуальный осмотр территории,

Document 2:
лицу или уполномоченному представителю юридического лица, индивидуальному предпринимателю, уполномоченному представителю индивидуального предпринимателя под расписку об ознакомлении либо об отказе в ознаком

In [8]:
import os
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    answer_correctness
)
os.environ["OPENAI_API_KEY"] = "sk-or-vv-678cbad362790622377958c8beb4937642cfb7083ee6cfce2d62886dc4b86326"
os.environ["OPENAI_API_BASE"] = "https://api.vsegpt.ru/v1"

In [12]:
class Evaluator:
    """
    Class for evaluating pipeline on test set.
    """
    def __init__(self, test_set_path: str):
        self.raw_test_data = pd.read_excel(test_set_path).drop(columns=['contexts', 'evolution_type', 'metadata', 'episode_done'])
        self.test_data = self.raw_test_data.sample(n=50, random_state=42)
        print(f"Test data size {self.test_data.shape}")

    def evaluate_pipe(self, pipeline) -> None:
        """
        Evaluate pipeline on test set.
        """
        answers = []
        contexts = []
        times = []
        for item in tqdm(self.test_data['question'].values):
            start_time = time.time()
            result = pipeline(item)
            end_time = time.time()
            answers.append(result['answer'])
            contexts.append(result['context'])
            times.append(end_time - start_time)

        mean_time = sum(times) / len(times)
        print(f"Mean time for getting answer: {mean_time}")

        test_answers = self.test_data
        test_answers['answer'] = answers
        test_answers['contexts'] = contexts
        test_answers.to_csv("answers.csv")
        dataset = Dataset.from_dict(test_answers.to_dict(orient='list'))

        result = evaluate(
            dataset,
            metrics=[
                context_precision,
                faithfulness,
                answer_relevancy,
                context_recall,
                answer_correctness
            ],
        )
        df = result.to_pandas()
        df['mean_time'] = mean_time
        df.to_csv(f'results_{pipeline.__class__.__name__}.csv')
        print(f"Saved results to 'results_{pipeline.__class__.__name__}.csv'")
        print(f"Metrics: {result}")

In [13]:
evaler = Evaluator("../data/v2_ragas_npa_dataset_firstPart.xlsx")

Test data size (50, 2)


In [14]:
def get_answer(question):
  retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 8},
  )
  relevant_docs = retriever.invoke(question)

  # request(localhost.com)

  # print("\n--- Relevant Documents ---")
  cont = []
  for i, doc in enumerate(relevant_docs, 8):
      cont.append(doc.page_content)

  combined_input = (
      "Вот несколько документов, которые могут помочь ответить на вопрос: "
      + question
      + "\n\nСоответствующие документы:\n"
      + "\n\n".join([doc.page_content for doc in relevant_docs])
      + "\n\nПожалуйста, предоставьте ответ, основываясь только на предоставленных документах. Если ответ не найден в документах, ответьте 'Я не уверен'."
  )

  model = ChatOpenAI(model="gpt-4o-mini", openai_api_key="sk-or-vv-678cbad362790622377958c8beb4937642cfb7083ee6cfce2d62886dc4b86326", base_url="https://api.vsegpt.ru/v1")

  messages = [
      SystemMessage(content="Ты ассистент отвечающий на вопросы по информации которая есть в нормативно-правовых-актах"),
      HumanMessage(content=combined_input),
  ]

  # Invoke the model with the combined input
  result = model.invoke(messages)
  return {"answer": result.content, "context": cont}

In [15]:
evaler.evaluate_pipe(get_answer)

100%|██████████| 50/50 [03:43<00:00,  4.46s/it]


Mean time for getting answer: 4.459153871536255


Evaluating:  20%|█▉        | 49/250 [01:26<02:17,  1.47it/s]Exception raised in Job[73]: BadRequestError(Error code: 400 - {'error': {'message': 'Potentially out of budget: 46348->700, expected price 0.98296, but you have only -2.352279 on account. Please, add some money to balance to proceed: https://vsegpt.ru/User/Money', 'code': 400}})
Exception raised in Job[0]: BadRequestError(Error code: 400 - {'error': {'message': 'Potentially out of budget: 9679->700, expected price 0.24958, but you have only -2.352279 on account. Please, add some money to balance to proceed: https://vsegpt.ru/User/Money', 'code': 400}})
Evaluating:  20%|██        | 51/250 [01:27<01:53,  1.76it/s]Exception raised in Job[135]: BadRequestError(Error code: 400 - {'error': {'message': 'Potentially out of budget: 11015->700, expected price 0.2763, but you have only -2.352279 on account. Please, add some money to balance to proceed: https://vsegpt.ru/User/Money', 'code': 400}})
Exception raised in Job[215]: BadReques

Saved results to 'results_function.csv'
Metrics: {'context_precision': 0.9747, 'faithfulness': 0.5464, 'answer_relevancy': 0.6543, 'context_recall': 0.8889, 'answer_correctness': 0.5396}
